In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import pingouin as pg
import plotly.express as px
from plotly.express.colors import n_colors
import plotly.graph_objects as go
from os.path import join as pjoin
from natsort import natsort_keygen, natsorted

sys.path.append("../")
import circletrack_behavior as ctb
import plotting_functions as pf
import circletrack_neural as ctn

In [ ]:
## Settings
project_folder = ['CircleTrack_Updating']
experiment_folders = ['Reversal2']
dpath = f'../../{project_folder[0]}'
fig_path = f'../../../Manuscripts/MemoryUpdating/intermediate_plots'
chance_color = 'darkgrey'
avg_color = 'midnightblue'
subject_color = 'darkgrey'
two_colors = ['darkorchid', 'midnightblue']
error_color = ['rgba(153,50,204,0.4)', 'rgba(25,25,112,0.4)'] #'rgba(169,169,169,0.4)'
session_list = ['A1', 'A2', 'A3', 'A4', 'A5', 'AU1', 'AU2', 'AU3', 'AU4', '', 'AR1', 'AR2'] ## space added for empty plot
old_new_list = ['AU1', 'AU2', 'AU3', 'AU4', 'AR1', 'AR2']

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

### Plot weight across days as a percentage of baseline weight.

In [ ]:
weight_data = pd.read_csv(pjoin(fig_path, 'intermediate_data/weight_from_baseline.csv'))
column_list = [f'{x}' for x in np.arange(1, 19)]
weight_melt = weight_data.melt(id_vars=['Mouse'], value_vars=column_list, var_name='day', value_name='percent_change')
weight_melt['day'] = weight_melt['day'].astype(int)
avg_weight = weight_melt.groupby(['day'], as_index=False).agg({'percent_change': ['mean', 'sem']})

fig = pf.custom_graph_template(x_title='Day', y_title='Weight Change from Baseline (%)')
fig.add_trace(go.Scatter(x=natsorted(avg_weight['day']), y=avg_weight['percent_change']['mean'], mode='lines+markers', line_color=avg_color,
                         error_y=dict(type='data', array=avg_weight['percent_change']['sem']), showlegend=False))

fig.update_yaxes(range=[75, 101])
fig.add_vline(x=7.5, line_width=1, opacity=1, line_dash='dash', line_color=chance_color)
fig.add_hline(y=80, line_width=1, opacity=1, line_dash='dash', line_color=chance_color)
fig.show()
fig.write_image(pjoin(fig_path, 'weight_from_baseline.png'))

### Plot fluorescence intensity across days for the experiment.

In [ ]:
fluo_data = pd.read_csv(pjoin(fig_path, 'intermediate_data/fluorescence_data.csv'))
avg_fluo = fluo_data.groupby(['day'], as_index=False).agg({'mean_fluorescence': ['mean', 'sem']})

fig = pf.custom_graph_template(x_title='Day', y_title='Mean Fluorescence (a.u.)')
fig.add_trace(go.Scatter(x=natsorted(avg_fluo['day']), y=avg_fluo['mean_fluorescence']['mean'], mode='lines+markers', line_color=avg_color,
                         error_y=dict(type='data', array=avg_fluo['mean_fluorescence']['sem']), showlegend=False))
for mouse in fluo_data['mouse'].unique():
    mdata = fluo_data[fluo_data['mouse'] == mouse]
    fig.add_trace(go.Scatter(x=mdata['day'], y=mdata['mean_fluorescence'], mode='lines', line_color=subject_color, 
                             name=mouse, opacity=0.7, line=dict(width=1), showlegend=False))
fig.show()

### Load S and YrA to see how well S matches YrA.

In [ ]:
mouse = 'rv03'
experiment = experiment_folders[0]
sess = 5
s_session = f'{mouse}_S_{sess}.nc'
yra_session = f'{mouse}_YrA_{sess}.nc'
rpath = pjoin(dpath, f'{experiment}/output/aligned_minian/{mouse}')

S = xr.open_dataset(pjoin(rpath, f'S/{s_session}'))['S']
YrA = xr.open_dataset(pjoin(rpath, f'YrA/{yra_session}'))['YrA']

In [ ]:
cell_id = 3
scale_factor = 3
fig = pf.custom_graph_template(x_title='Time (s)', y_title='Activity (a.u.)', width=1000)
fig.add_trace(go.Scatter(x=YrA['behav_t'], y=YrA.values[cell_id], mode='lines', line_color='darkgrey', name='dF/F'))
fig.add_trace(go.Scatter(x=S['behav_t'], y=S.values[cell_id]*scale_factor, mode='lines', 
                         line_color='red', name='S', opacity=0.7))
fig.show()
fig.write_image(pjoin(fig_path, 'S_over_fluorescence.png'))

### Loading sessions for testing alignments.

In [ ]:
experiment = experiment_folders[0]
offset = 5
col_name = 'unix'
starting_idx = 0
mouse = 'rv03'
date = '2024_11_17'
timestamp = '10_12_01'
behav_path = pjoin(dpath, f'{experiment}/output/behav/{mouse}')
behav = pd.read_feather(pjoin(behav_path, f'{mouse}_{2}.feat'))
act = ctn.open_minian(f'/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrack_Updating/Reversal2/minian_results/{mouse}/{date}/{timestamp}/minian')['S']
act_shifted, aligned_behav = ctn.align_calcium_behavior(act, behav, col_name=col_name)